In [47]:
import numpy as np

In [48]:
class SequenceAlignment:
    
    def __init__(self,s1,s2,match=2,mismatch=1,penalty=0):
        self.s1=s1
        self.s2=s2
        self.m=len(self.s1)+1
        self.n=len(self.s2)+1
        self.A=None
        self.T=None
        self.penalty=penalty
        self.match=match
        self.mismatch=mismatch
    
    def initialise(self):
        self.A=np.zeros((self.m,self.n))
        self.T=np.zeros((self.m,self.n))
        
        if (self.penalty>0):
            self.A[0]=[i for i in range(0,-self.penalty*self.m,-self.penalty)]
            self.A[:,0]=[i for i in range(0,-self.penalty*self.m,-self.penalty)]
        
            self.T[0]=[1 for i in range(self.m)]
            self.T[:,0]=[2 for i in range(self.n)]
        
            self.T[0][0]=0
            
    def matrices(self):
        for i in range(1,self.n):
            for j in range(1,self.m):

                #match
                if(self.s2[i-1]==self.s1[j-1]):
                    self.A[i][j]=self.match+self.A[i-1][j-1]
                    self.T[i][j]= 3

                #mismatch
                elif(self.s2[i-1]!=self.s1[j-1]):
                    
                    gap_open_penalty=-self.penalty
                    
                    diagonal=self.A[i-1][j-1]-self.mismatch
                    up=self.A[i-1][j]+gap_open_penalty
                    left=self.A[i][j-1]+gap_open_penalty
                    
                    #print(self.s2[i-1],self.s1[j-1],diagonal,up,left)
                    
                    num,direction=Maxx(diagonal,up,left)
                    
                    self.A[i][j]=num
                    
                    self.T[i][j]=direction
                    
    def get_score(self):
        i=len(self.s2)
        j=len(self.s1)
        score=self.A[i][j]
        
        return i,j, score
                    
    def get_sequences(self,i,j,seq1,seq2):
        
        if(i==0 and j==0):
            return seq1,seq2
        
        
        if self.T[i][j]==3: #diagonal
            
            if(self.s1[j-1]==self.s2[i-1]):
                seq1=self.s1[j-1]+seq1
                seq2=self.s2[i-1]+seq2
                
            elif(self.s1[j-1]!=self.s2[i-1]):
                seq1=self.s1[j-1]+seq1
                seq2=self.s2[i-1]+seq2
            
            i-=1
            j-=1
            
        elif self.T[i][j]==2:
            
            seq1="_"+seq1
            seq2=self.s2[i-1]+seq2
            i-=1
        elif self.T[i][j]==1:
            
            seq1=self.s1[j-1]+seq1
            seq2='_'+seq2
            j-=1
            
            
        return self.get_sequences(i,j,seq1,seq2)
        

    def printmat(self):
        for i in range(self.m):
            for j in range(self.n):
                print(self.A[i][j],end=" ")
            print("\n")
            
    def maximum(self,x,y,z):
        if(x>y and x>z):
            return x
        elif(y>x and y>z):
            return y
        else:
            return z
        
    def alignment_score(self):
        
        i, j, mat_score = sa.get_score()
        s1,s2=sa.get_sequences(i,j,'','')
        
        score=0

        for i in range(len(s1)):

            if(s1[i]==s2[i]): #match
                score+=self.match 
                print("match :",self.match )
            elif(s1[i]=='_' or s2[i]=='_'): #gap penalty
                score-=self.penalty
                print("penalty :",self.penalty )
            elif(s1[i]!=s2[i]): #mismatch!
                score-=self.mismatch
                print("mismatch :",self.mismatch )
        return score

def Maxx(x,y,z):
    
    if(x>y and x>z):
        return (x,3)
    
    elif(y>x and y>z):
        return (y,2)
    
    elif(z>x and z>y):
        return (z,1)
    
    else:
        return x,3
    

In [49]:
s1='TGGTG'
s2='ATCGT'

sa=SequenceAlignment(s1,s2,penalty=2,match=1,mismatch=1)
sa.initialise()
sa.matrices()
sa.printmat()

0.0 -2.0 -4.0 -6.0 -8.0 -10.0 

-2.0 -1.0 -3.0 -5.0 -7.0 -9.0 

-4.0 -1.0 -2.0 -4.0 -4.0 -6.0 

-6.0 -3.0 -2.0 -3.0 -5.0 -5.0 

-8.0 -5.0 -2.0 -1.0 -3.0 -4.0 

-10.0 -7.0 -4.0 -3.0 0.0 -2.0 



In [50]:
sa.T

array([[0., 1., 1., 1., 1., 1.],
       [2., 3., 3., 3., 3., 3.],
       [2., 3., 3., 3., 3., 1.],
       [2., 2., 3., 3., 3., 3.],
       [2., 2., 3., 3., 1., 3.],
       [2., 3., 2., 3., 3., 1.]])

In [51]:
i, j, score = sa.get_score()
ss1,ss2=sa.get_sequences(i,j,'','')

In [52]:
score

-2.0

In [53]:
ss1,ss2

('_TGGTG', 'ATCGT_')

In [54]:
sa.alignment_score()

penalty : 2
match : 1
mismatch : 1
match : 1
match : 1
penalty : 2


-2

In [55]:
s1='ACACACTA'
s2='AGCACACA'

sa=SequenceAlignment(s1,s2,penalty=1,match=2,mismatch=1)
sa.initialise()
sa.matrices()
sa.printmat()

0.0 -1.0 -2.0 -3.0 -4.0 -5.0 -6.0 -7.0 -8.0 

-1.0 2.0 1.0 0.0 -1.0 -2.0 -3.0 -4.0 -5.0 

-2.0 1.0 1.0 0.0 -1.0 -2.0 -3.0 -4.0 -5.0 

-3.0 0.0 3.0 2.0 2.0 1.0 0.0 -1.0 -2.0 

-4.0 -1.0 2.0 5.0 4.0 4.0 3.0 2.0 1.0 

-5.0 -2.0 1.0 4.0 7.0 6.0 6.0 5.0 4.0 

-6.0 -3.0 0.0 3.0 6.0 9.0 8.0 7.0 7.0 

-7.0 -4.0 -1.0 2.0 5.0 8.0 11.0 10.0 9.0 

-8.0 -5.0 -2.0 1.0 4.0 7.0 10.0 10.0 12.0 



In [56]:
i, j, score = sa.get_score()
ss1,ss2=sa.get_sequences(i,j,'','')
ss1,ss2

('A_CACACTA', 'AGCACAC_A')

In [57]:
sa.alignment_score()

match : 2
penalty : 1
match : 2
match : 2
match : 2
match : 2
match : 2
penalty : 1
match : 2


12